# Let's try to evaluate a random baseline 

In [ ]:
from pathlib import Path
import pandas as pd 
import numpy as np
import altair as alt
from dask.distributed import Client

In [ ]:
%load_ext autoreload
%autoreload 2

## Read the data

In [ ]:
from energyclustering.data.fluvius.data import read_data_pickle
from energyclustering.data.weather.data import read_weather_data
from energyclustering.sampling.preprocessing import preprocess_info_df_for_sampling_classification

In [ ]:
info_df, data_df = read_data_pickle()
info_df = info_df.set_axis(info_df.index.to_flat_index().map(str), axis = 0)
data_df = data_df.set_axis(data_df.index.to_flat_index().map(str), axis = 0)

In [ ]:
# data_df = data_df.sample(30)
# info_df = info_df.loc[data_df.index]

In [ ]:
info_df = preprocess_info_df_for_sampling_classification(info_df, data_df)

In [ ]:
generator = np.random.default_rng(1)
shuffled = data_df.index.to_numpy(copy=True)
generator.shuffle(shuffled)
folds = np.array_split(shuffled, 3)

In [ ]:
print(f"data_df = {data_df.memory_usage().sum() / 1000 / 1000} MB")
print(f"info_df = {info_df.memory_usage().sum() / 1000 / 1000} MB")

In [ ]:
weather_data = (
    read_weather_data('aalst')
    .drop(columns = ['moon_illumination', 'moonrise', 'moonset', 'sunrise', 'winddirDegree', 'location', 'DewPointC', 'sunset'])
    .set_index('date_time')
)
weather_data.head()

In [ ]:
print(f"weather_df = {weather_data.memory_usage().sum() / 1000 / 1000} MB")

## Preprocess the data

In [ ]:
import energyclustering.sampling.preprocessing as pre

In [ ]:
%%time
daily_data_df = pre.yearly_profile_df_to_daily_df(data_df)

In [ ]:
%%time
daily_info_df = pre.to_daily_metadata_df(info_df, weather_data)

### Remove all NaN days from data and info df

In [ ]:
contains_nan_day = daily_data_df.isna().any(axis = 1)
daily_data_df = daily_data_df.loc[~contains_nan_day]
daily_info_df = daily_info_df.loc[~contains_nan_day]

In [ ]:
daily_data_df.head()

In [ ]:
daily_info_df

In [ ]:
print(f"daily_info_df = {daily_info_df.memory_usage().sum() / 1000 / 1000} MB")
print(f"daily_data_df = {daily_data_df.memory_usage().sum() / 1000/1000} MB")

In [ ]:
weather_data

In [ ]:
nb_days = int(daily_data_df.shape[0]*2/7)

In [ ]:
daily_data_df = daily_data_df.sample(nb_days, random_state = 1234)
daily_info_df = daily_info_df.loc[daily_data_df.index].rename_axis(('meterID', 'date'))

## Sampler

In [ ]:
from energyclustering.sampling.samplers import MetadataSampler
from energyclustering.sampling.day_of_year_samplers import RandomDayFromYearSampler, SpecificDayFromYearSampler, SimilarDayFromYearSampler
from sklearn.cluster import KMeans

# Experimental set-up 

## Step 1: subsample days 

## Step 2: ablation study on energyville current approach
We know the energy score for totally random sampling. 
So let's see if we can improve upon that with the current energyville procedure.
- first comparison: totally random vs pick the year first through metadata clustering --> this way we can measure if the metadata clustering is helpful 
- second comparison: totally random vs random year but day sampled with similar weather --> is taking similar weather day beneficial 
- third comparison: previous three vs metadata + similar weather --> to see the joint effect 

## Evaluator

In [ ]:
from energyclustering.sampling.evaluation import DaskEnergyScoreEvaluator, LocalEnergyScoreEvaluator, train_test_sets_from_folds
from pathlib import Path

In [ ]:
result_path = Path().absolute() / 'results'

In [ ]:
with Client(local_directory = '/cw/dtailocal/', n_workers=20, threads_per_worker = 1) as client: 
    evaluator = DaskEnergyScoreEvaluator(folds, daily_data_df, daily_info_df, data_df, client, 200)
    sampler = SimilarDayFromYearSampler(MetadataSampler(KMeans(n_clusters = 10)), 10, weather_data)
    evaluator.evaluate_and_save(sampler, result_path/'metadata+weather.pkl')
  

## Show results


In [ ]:
result_df = pd.concat([pd.read_pickle(result_path/f"random_{number}.pkl").rename(number) for number in [10, 50, 100, 250, 500, 1000]], axis = 1)
result_df

In [ ]:
plot_df = result_df.agg(['mean', 'std'], axis =0).T.rename_axis('nb_samples', axis = 0).reset_index().assign(lower = lambda x: x['mean']-x['std'], higher = lambda x: x['mean'] + x['std'])
plot_df

In [ ]:
line = alt.Chart(plot_df, title = 'energy score of random sampling with limited samples').mark_line().encode(
    x = 'nb_samples:Q', 
    y = alt.Y('mean:Q', title = 'mean energy score', scale = alt.Scale(zero = False)),
)
line